In [1]:
import cv2
import ast
import mmcv
import math
import torch
import torch.nn.functional as F
import torchvision
import pickle as pkl
import pandas as pd
import numpy as np
import seaborn as sns
from einops import rearrange
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

from torchmetrics.functional.classification import (
    multilabel_f1_score,
    multilabel_precision,
    multilabel_recall,
)

from data_utils import results2df
from sklearn.metrics.pairwise import cosine_similarity

# Slowfast imports
from slowfast.models import build_model
from slowfast.utils.parser import load_config, alt_parse_args
from slowfast.datasets.loader import construct_loader

/home/dl18206/anaconda3/envs/slowfast/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/dl18206/anaconda3/envs/slowfast/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
# Set random seed for reproducibility
torch.manual_seed(0)

**Generating few-shot annotation file**

In [3]:
train_path = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/annotations/standard/fg_only/standard/train.csv"
val_path = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/annotations/standard/fg_only/standard/val.csv"
metadata_path = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/metadata/with_negative_pairing/new_metadata.csv"

train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)

train_df.columns = ["subject_id", "label"]
val_df.columns = ["subject_id", "label"]

metadata = pd.read_csv(metadata_path)

with open(
    "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/metadata/behaviours.txt",
    "rb",
) as f:
    behaviours = [beh.decode("utf-8").strip() for beh in f.readlines()]

train_df = train_df.merge(
    metadata[["subject_id_fg", "value"]], right_on="subject_id_fg", left_on="subject_id"
)

val_df = val_df.merge(
    metadata[["subject_id_fg", "value"]], right_on="subject_id_fg", left_on="subject_id"
)


def is_fs(x):
    fs_behaviours = ["aggression"]
    for b in fs_behaviours:
        if b == x:
            return True
    for b in fs_behaviours:
        if b in x.split(","):
            return True
    return False


train_df[
    (train_df["value"].apply(is_fs)) & (train_df.subject_id.str.startswith("acp"))
][["subject_id", "label", "value"]]

val_df[(val_df["value"].apply(is_fs))][["subject_id", "label", "value"]]

,subject_id,label,value
8,60910995.mp4,"[1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]","display,aggression,piloerection,climbing,travel"
15,36070556.mp4,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]","aggression,display,vocalisation"
20,36070537.mp4,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1]","resting,travel,vocalisation,display,aggression"
24,36070523.mp4,"[1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1]","display,aggression,object_carrying,bipedal,cli..."
37,59759678.mp4,"[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","climbing,aggression,display"
54,36072978.mp4,"[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]","aggression,object_carrying,display"
59,36072994.mp4,"[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]","display,vocalisation,aggression,climbing,travel"
62,36072986.mp4,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]","aggression,vocalisation,display,travel"
68,36072949.mp4,"[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1]","display,bipedal,object_carrying,aggression,voc..."
78,32836460.mp4,"[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]","camera_reaction,aggression,piloerection,vocali..."


In [4]:
val_df_neg = pd.read_csv(
    "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/dataset/annotations/standard/with_negative_pairing/standard/val.csv"
)
val_df_neg.columns = ["fg", "bg", "label", "negative", "utm_code"]
val_df_neg = val_df_neg.merge(
    metadata[["subject_id_fg", "value"]], right_on="subject_id_fg", left_on="fg"
)[["fg", "bg", "label", "value"]]

fs_val_df_neg = val_df_neg[val_df_neg["value"].apply(is_fs)]
fs_labels = fs_val_df_neg.label

**Temporal activation maps**

In [5]:
# Load the model
path_to_config = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/configs/SLOW_8x8_R50_Local.yaml"
path_to_ckpt = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/checkpoint_epoch_00100.pyth"

args = alt_parse_args()[:-1]
cfg = load_config(
    args[0],
    path_to_config=path_to_config,
)
checkpoint = torch.load(path_to_ckpt, map_location="cpu")

model = build_model(cfg)
model.load_state_dict(checkpoint["model_state"])
model.eval()
model.cpu()

classifier = model.head.projection

# Load the data
loader = construct_loader(cfg, "test")  # dataset = build_dataset("tap", cfg, "train")
inputs, labels, index, time, meta = next(iter(loader))

SAMPLING.BALANCED: False; BALANCE_TYPE: None


/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/slowfast/datasets/decoder.py:335: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  video_tensor = torch.from_numpy(np.frombuffer(video_handle, dtype=np.uint8))
/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/slowfast/datasets/decoder.py:335: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for t

**Funcs**

In [6]:
def get_feature_map(model, sample):
    with torch.no_grad():
        feature_map = model.s5(
            model.s4(model.s3(model.s2(model.s1([sample.unsqueeze(0)]))))
        )[
            0
        ]  # TODO: Investigate features at earlier layers
    return feature_map


def extract_frame_wise_features(feature_map, t):
    spatially_pooled = F.adaptive_avg_pool3d(feature_map, (t, 1, 1))
    frame_wise_features = torch.flatten(spatially_pooled, start_dim=2)
    return frame_wise_features

In [9]:
# Select a sample
sample_index = 0
label = labels[sample_index]
sample = inputs[0][sample_index]
negative_sample = inputs[0][-1]

In [8]:
# Get feature maps for fg and bg
feature_map = get_feature_map(model, sample)
negative_feature_map = get_feature_map(model, negative_sample)
framewise_features = extract_frame_wise_features(feature_map, t=16)
negative_framewise_features = extract_frame_wise_features(negative_feature_map, t=16)

In [9]:
# Compute frame-wise cosine similarity for
normalise = False
framewise_features = framewise_features.squeeze(0)
framewise_features = framewise_features.T
if normalise:
    framewise_features = F.normalize(framewise_features, p=2, dim=1)
framewise_cosine_similarity = torch.mm(framewise_features, framewise_features.T)

In [10]:
frame_wise_logits = []
for feat in framewise_features:
    frame_wise_logits.append(torch.sigmoid(classifier(feat).detach()).numpy())
frame_wise_logits = np.array(frame_wise_logits)

In [11]:
# Video feats and logits
video_level_features = F.adaptive_avg_pool3d(feature_map, (1, 1, 1))
video_level_features = torch.flatten(video_level_features, start_dim=1)
video_level_logits = torch.sigmoid(classifier(video_level_features)).detach().numpy()
video_level_logits = np.array(video_level_logits)

In [ ]:
# Plot cosine similarity matrix
plt.figure(figsize=(12, 12))
sns.heatmap(framewise_cosine_similarity.detach().numpy(), cmap="viridis", annot=True)
plt.title("Frame-wise cosine similarity")
plt.show()

In [ ]:
# Plot frame-wise logits as heatmap
plt.figure(figsize=(12, 8), dpi=100)
sns.heatmap(frame_wise_logits.T, cmap="viridis", annot=True, yticklabels=behaviours)
plt.title("Frame-wise logits")
plt.show()

In [ ]:
# Plot framewise logits and video-level logits side by side
fig, ax = plt.subplots(
    1, 3, figsize=(20, 10), gridspec_kw={"width_ratios": [3, 0.75, 0.75]}
)
sns.heatmap(
    frame_wise_logits.T, cmap="viridis", annot=True, yticklabels=behaviours, ax=ax[0]
)
sns.heatmap(
    video_level_logits.T,
    cmap="viridis",
    annot=True,
    ax=ax[1],
    cbar=False,
)
sns.heatmap(
    label.unsqueeze(0).numpy().T,
    cmap="viridis",
    annot=True,
    ax=ax[2],
    cbar=False,
)

ax[0].title.set_text("Masked Frame-wise Logits")
ax[1].title.set_text("Video-level Logits")
ax[2].title.set_text("Label")

plt.show()

In [15]:
subframe_features = framewise_features[0:8:,]

# Subframe logits
subframe_logits = []
for feat in subframe_features:
    subframe_logits.append(torch.sigmoid(classifier(feat).detach()).numpy())

subframe_logits = np.array(subframe_logits)

# Adaptive pool over subframe features
subframe_features = subframe_features.unsqueeze(0)
subframe_features = F.adaptive_avg_pool2d(subframe_features, (1, 2048))[0]

# Apply classifier to subframe features
subvideo_logits = torch.sigmoid(classifier(subframe_features)).detach().numpy()

In [ ]:
# Plot framewise logits and video-level logits side by side
fig, ax = plt.subplots(
    1, 4, figsize=(20, 10), gridspec_kw={"width_ratios": [3, 0.75, 0.75, 0.75]}
)
sns.heatmap(
    subframe_logits.T, cmap="viridis", annot=True, yticklabels=behaviours, ax=ax[0]
)
sns.heatmap(
    subvideo_logits.T,
    cmap="viridis",
    annot=True,
    ax=ax[1],
    cbar=False,
)

sns.heatmap(
    video_level_logits.T,
    cmap="viridis",
    annot=True,
    ax=ax[2],
    cbar=False,
)

sns.heatmap(
    label.unsqueeze(0).numpy().T,
    cmap="viridis",
    annot=True,
    ax=ax[3],
    cbar=False,
)

ax[0].title.set_text("Masked Frame-wise Logits")
ax[1].title.set_text("Sub-video Logits")
ax[2].title.set_text("Video-level Logits")
ax[3].title.set_text("Label")

plt.show()